In [14]:
from pathlib import Path
import numpy as np
import random
import struct

N_IMG=1000
N_DIM=128
N_CLUSTERS=8
ITER_K_MEANS=50
N_RAND=100000
MAX_DIM=N_DIM*N_CLUSTERS #1024
N_SEED=4000
random.seed(N_SEED)

In [3]:
data=[]
for num in range(N_IMG):
    features=[item for item in Path(f'./feats/{str(num).zfill(5)}.sift').read_bytes()]
    data.extend(np.reshape(features,(-1,N_DIM)))
data=np.array(data)
data.shape #(702120, 128)

(702120, 128)

In [4]:
def get_rand_array(array,num):
    randIdx=np.random.choice(len(array),num,replace=False)
    return array[randIdx]

In [5]:
def L2_distance(feature,cluster):
    return np.sqrt(np.sum((feature-cluster)**2))

In [6]:
def predict_cluster(feature,clusters):
    d=[L2_distance(feature,cluster) for cluster in clusters]
    return np.argmin(d)

In [7]:
def init_kmeans_plus(data):
    clusters=[random.choice(data)]
    for _ in range(N_CLUSTERS-1):
        distance=np.array([min([L2_distance(feature,cluster) for cluster in clusters]) for feature in data])
        cluster=data[np.argmax(distance)]
        clusters.append(cluster)
    return clusters

In [8]:
def cluster_centers_kmeans(data,clusters):

    for i in range(ITER_K_MEANS):
        print(f"[{i}/{ITER_K_MEANS}] clusters: {np.sum(clusters)}")
        tmpClusters=np.zeros((N_CLUSTERS,N_DIM))
        count=[0]*N_CLUSTERS

        for feature in data:
            mini=predict_cluster(feature,clusters)
            count[mini]+=1
            tmpClusters[mini]+=feature

        for cluster,cnt in zip(tmpClusters,count):
            if cnt==0:
                print(f"cnt==0")
    #         tmpClusters[j]=clusters[j]
            cluster/=cnt
        clusters=tmpClusters
    print(f"[result] clusters:{np.sum(clusters)}")

    return clusters

In [9]:
rand10e4=get_rand_array(data,N_RAND)
np.save(f'rand{N_SEED}',rand10e4)
rand10e4=np.load(f'rand{N_SEED}.npy')

In [10]:
clusters=np.array(init_kmeans_plus(rand10e4)) #k-means++
np.save(f'kmeans++_{N_SEED}', clusters)
clusters=get_rand_array(rand10e4,N_CLUSTERS) #k-means
np.save(f'kmeans_{N_SEED}', clusters)
clusters=np.load(f'kmeans++_{N_SEED}.npy')

In [11]:
clusters=cluster_centers_kmeans(rand10e4,clusters)

[0/50] clusters: 59478
[1/50] clusters: 62024.58856384285
[2/50] clusters: 63176.87813688468
[3/50] clusters: 63464.471828596106
[4/50] clusters: 63684.877654056036
[5/50] clusters: 63814.30045697787
[6/50] clusters: 63880.22200657037
[7/50] clusters: 63917.32610254726
[8/50] clusters: 63893.28848656936
[9/50] clusters: 63808.9120531044
[10/50] clusters: 63674.65662374483
[11/50] clusters: 63499.352800225315
[12/50] clusters: 63311.66169677315
[13/50] clusters: 63136.158603524396
[14/50] clusters: 62969.74976747481
[15/50] clusters: 62828.23665458588
[16/50] clusters: 62695.088403862
[17/50] clusters: 62591.65478566327
[18/50] clusters: 62511.56356471559
[19/50] clusters: 62447.440289620965
[20/50] clusters: 62380.55957182484
[21/50] clusters: 62310.27106435543
[22/50] clusters: 62254.584080301895
[23/50] clusters: 62201.75833811077
[24/50] clusters: 62164.630564913496
[25/50] clusters: 62130.0517010967
[26/50] clusters: 62095.46965233595
[27/50] clusters: 62069.38728383357
[28/50] clu

In [ ]:
def get_descriptor(data,clusters):
    vlad=np.zeros((MAX_DIM))
    for feature in data:
        label=predict_cluster(feature,clusters)
        start=label*N_DIM
        end=start+N_DIM
        vlad[start:end]+=feature-clusters[label]
    vlad=np.sign(vlad)*np.sqrt(np.abs(vlad))
    vlad/= np.sqrt(np.sum(vlad**2)) 
    return vlad

In [ ]:
def get_features_from_img(IMG_PATH):
    features=[item for item in Path(f'./feats/{IMG_PATH}.sift').read_bytes()]
    return np.reshape(features,(-1,N_DIM))

In [12]:
descriptor = np.zeros((N_IMG, MAX_DIM))
for num in range(N_IMG):
    IMG_PATH=str(num).zfill(5)
    data=[item for item in Path(f'./feats/{IMG_PATH}.sift').read_bytes()]
    data=np.reshape(data,(-1,N_DIM))
    descriptor[num]=get_descriptor(data,clusters)

In [13]:
with open(f'kmeansP_{N_SEED}.des', 'wb') as f:
    f.write(struct.pack('ii', N_IMG, MAX_DIM))
    f.write(descriptor.astype('float32').tobytes())